In [182]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
pd.set_option('display.max_columns', None)

In [183]:
features = pd.read_csv('../data/raw/feature_definitions.csv')
features.head(2)

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
1,actualdpdtolerance_344P,DPD of client with tolerance.


In [184]:
debitcard = pd.read_parquet('../data/raw/train_debitcard_1.parquet')
debitcard.head()

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
0,225,NaN,NaN,NaN,0,2016-08-16
1,331,NaN,NaN,NaN,0,2015-03-19
2,358,NaN,NaN,NaN,0,2014-09-02
3,390,NaN,NaN,NaN,0,2014-07-23
4,390,NaN,NaN,NaN,2,2016-06-08


In [185]:
features[features['Variable'].isin(debitcard.columns)]

,Variable,Description
226,last180dayaveragebalance_704A,Average balance on debit card in the last 180 ...
227,last180dayturnover_1134A,Debit card's turnover within the last 180 days.
228,last30dayturnover_651A,Debit card turnover for the last 30 days.
337,openingdate_857D,Debit card opening date.


In [186]:
debitcard.count()

case_id                          157302
last180dayaveragebalance_704A     12216
last180dayturnover_1134A          11081
last30dayturnover_651A            11081
num_group1                       157302
openingdate_857D                 144591
dtype: int64

In [187]:
last_case_acceptance_date = '2020-10-05' # source: isuru's cleaning job

debit cards

In [188]:
debitcard_valid = debitcard.dropna()
debitcard_valid['debitcard_days'] = (pd.to_datetime(last_case_acceptance_date) - pd.to_datetime(debitcard_valid['openingdate_857D'])).dt.days
debitcard_valid.head(2)



C:\Users\tharaka_50498.DIALOG\AppData\Local\Temp\1\ipykernel_2028\2045614203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  debitcard_valid['debitcard_days'] = (pd.to_datetime(last_case_acceptance_date) - pd.to_datetime(debitcard_valid['openingdate_857D'])).dt.days


,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D,debitcard_days
1287,103558,0.0,47400.0,0.0,0,2015-12-03,1768
1359,103827,0.0,38000.0,0.0,0,2015-12-14,1757


In [189]:
debitcard_mean = debitcard_valid.drop(columns=['openingdate_857D']).groupby('case_id').mean().reset_index()
debitcard_mean.drop(columns=['num_group1', 'last30dayturnover_651A'], inplace=True)
debitcard_mean.head(2)

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,debitcard_days
0,103558,0.0,47400.0,1768.0
1,103827,0.0,38000.0,1757.0


deposits

In [190]:
deposit = pd.read_parquet('../data/raw/train_deposit_1.parquet')
deposit.head()

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,None,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,None,0,2014-09-02
3,390,211748.53000,2017-07-22,0,2014-07-23
4,390,223.68001,None,2,2016-06-08


In [191]:
deposit.count()

case_id                 145086
amount_416A             145086
contractenddate_991D     65404
num_group1              145086
openingdate_313D        145086
dtype: int64

In [192]:
# filling the contract end date as today (sample collection) for active contracts. (goal is to find the days since expiry)
deposit.loc[deposit['contractenddate_991D'].isnull(), 'contractenddate_991D'] = last_case_acceptance_date

deposit['contract_days'] = (pd.to_datetime(deposit['contractenddate_991D']) - pd.to_datetime(deposit['openingdate_313D'])).dt.days
deposit.head(2)

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D,contract_days
0,225,0.000,2020-10-05,0,2016-08-16,1511
1,331,260.374,2018-03-18,0,2015-03-19,1095


In [193]:
#removing date columns
deposit.drop(columns=['contractenddate_991D', 'openingdate_313D', 'num_group1'], inplace=True)
deposit_mean = deposit.groupby('case_id').mean().reset_index()
deposit_mean.head(2)

,case_id,amount_416A,contract_days
0,225,0.000,1511.0
1,331,260.374,1095.0


person information

In [194]:
person_1 = pd.read_parquet('../data/raw/train_person_1.parquet')
person_1.head(10)

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,isreference_387L,language1_981M,mainoccupationinc_384A,maritalst_703L,num_group1,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,None,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,MORE_FIVE,OTHER,P142_57_166,P167_100_165,MARRIED,None,None,None,SALARIED_GOVT,None,P10_39_147,10800.0,None,0,0.0,1.0,1.0,P88_18_84,P167_100_165,None,None,None,CL,None,True,F,PRIMARY_MOBILE
1,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,1,1.0,1.0,4.0,a55475b1,a55475b1,SPOUSE,None,False,EM,None,None,None,PHONE
2,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,2,2.0,4.0,5.0,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,None,None,None,PHONE
3,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,3,NaN,5.0,NaN,a55475b1,a55475b1,None,COLLEAGUE,None,PE,None,None,None,PHONE
4,1,1957-08-01,None,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,MORE_FIVE,OTHER,P49_46_174,P160_59_140,DIVORCED,None,None,None,SALARIED_GOVT,None,P10_39_147,10000.0,None,0,0.0,1.0,1.0,P103_93_94,P176_37_166,None,None,None,CL,None,True,M,PRIMARY_MOBILE
5,1,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,1,1.0,1.0,5.0,a55475b1,a55475b1,SIBLING,None,False,CL,None,None,None,PRIMARY_EMAIL
6,1,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,2,2.0,1.0,5.0,a55475b1,a55475b1,OTHER_RELATIVE,None,False,EM,None,None,None,PHONE
7,1,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,3,NaN,5.0,NaN,a55475b1,a55475b1,None,SIBLING,None,PE,None,None,None,PHONE
8,1,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,None,None,a55475b1,a55475b1,None,None,None,None,None,None,a55475b1,NaN,None,4,NaN,5.0,NaN,a55475b1,a55475b1,None,OTHER_RELATIVE,None,PE,None,None,None,PHONE
9,2,1974-12-01,None,NaN,P91_125_184,False,False,P161_5_97,P97_36_170,2010-02-15,MORE_FIVE,OTHER,P63_14_24,P161_5_97,MARRIED,None,None,None,EMPLOYED,None,P10_39_147,14000.0,None,0,0.0,1.0,1.0,P91_125_184,P161_5_97,None,None,None,EM,None,True,F,PRIMARY_MOBILE


In [195]:
person_1['empl_industry_691L'].unique()

array(['OTHER', None, 'EDUCATION', 'GOVERNMENT', 'MINING', 'FINANCE',
       'HEALTH', 'ART_MEDIA', 'WELNESS', 'TRADE', 'POST_TELCO',
       'TRANSPORTATION', 'MANUFACTURING', 'IT', 'AGRICULTURE', 'CATERING',
       'LAWYER', 'REAL_ESTATE', 'TOURISM', 'GAMING', 'CHARITY_RELIGIOUS',
       'INSURANCE', 'MARKETING', 'ARMY_POLICE', 'RECRUITMENT'],
      dtype=object)

In [196]:
features[features['Variable'].isin(person_1.columns)]

,Variable,Description
40,birth_259D,Date of birth of the person.
42,birthdate_87D,Birth date of the person.
47,childnum_185L,Number of children of the applicant.
77,contaddr_district_15M,Zip code of a contact person's address.
78,contaddr_matchlist_1032L,Indicates whether the contact address is found...
79,contaddr_smempladdr_334L,Indicates whether the contact address is the s...
80,contaddr_zipcode_807M,Zip code of contact address.
162,education_927M,Education level of the person.
164,empl_employedfrom_271D,Start date of employment.
165,empl_employedtotal_800L,Employment length of a person.


In [197]:
#selecting only numeric and simply encodable columns
person_selects = ['case_id', 'birth_259D', 'childnum_185L', 'education_927M', 'empl_employedtotal_800L', 'empl_industry_691L', 'familystate_447L', 'incometype_1044T', 'mainoccupationinc_384A']

In [198]:
person_1_selected = person_1[person_selects].copy()

In [199]:
# for now, we are only interested in the applicant. 
person_1_selected = person_1_selected[person_1_selected['birth_259D'].notnull()]
person_1_selected.count()

case_id                    1526659
birth_259D                 1526659
childnum_185L                    0
education_927M             1526659
empl_employedtotal_800L     528315
empl_industry_691L          522236
familystate_447L            728613
incometype_1044T           1526659
mainoccupationinc_384A     1526659
dtype: int64

In [200]:
person_1_selected['age'] = (pd.to_datetime(last_case_acceptance_date) - pd.to_datetime(person_1_selected['birth_259D'])).dt.days / 365
person_1_selected.drop(columns=['birth_259D', 'childnum_185L'], inplace=True)
person_1_selected.head(2)

,case_id,education_927M,empl_employedtotal_800L,empl_industry_691L,familystate_447L,incometype_1044T,mainoccupationinc_384A,age
0,0,P97_36_170,MORE_FIVE,OTHER,MARRIED,SALARIED_GOVT,10800.0,34.287671
4,1,P97_36_170,MORE_FIVE,OTHER,DIVORCED,SALARIED_GOVT,10000.0,63.221918


In [201]:
person_1_selected['empl_employedtotal_800L'].unique()

array(['MORE_FIVE', 'LESS_ONE', None, 'MORE_ONE'], dtype=object)

In [202]:
person_1_selected.loc[person_1_selected['empl_employedtotal_800L'].isnull(), 'empl_employedtotal_800L'] = 'NO'

In [203]:
encoding_map = {'NO': 0, 'LESS_ONE': 1, 'MORE_ONE': 2, 'MORE_FIVE': 3}
person_1_selected['emplyment_period_cat'] = person_1_selected['empl_employedtotal_800L'].map(encoding_map)
person_1_selected.drop(columns='empl_employedtotal_800L', inplace=True)
person_1_selected.head(2)


,case_id,education_927M,empl_industry_691L,familystate_447L,incometype_1044T,mainoccupationinc_384A,age,emplyment_period_cat
0,0,P97_36_170,OTHER,MARRIED,SALARIED_GOVT,10800.0,34.287671,3
4,1,P97_36_170,OTHER,DIVORCED,SALARIED_GOVT,10000.0,63.221918,3


In [204]:
person_1_selected['familystate_447L'].unique()

array(['MARRIED', 'DIVORCED', 'SINGLE', 'WIDOWED', 'LIVING_WITH_PARTNER',
       None], dtype=object)

In [205]:
person_1_selected[person_1_selected['familystate_447L'].isnull()].head()

,case_id,education_927M,empl_industry_691L,familystate_447L,incometype_1044T,mainoccupationinc_384A,age,emplyment_period_cat
119530,34639,a55475b1,None,None,RETIRED_PENSIONER,15600.0,57.553425,0
121959,35345,a55475b1,None,None,PRIVATE_SECTOR_EMPLOYEE,40000.0,45.043836,0
122700,35573,a55475b1,None,None,EMPLOYED,30000.0,38.372603,0
123087,35686,a55475b1,None,None,SALARIED_GOVT,60000.0,31.032877,0
123471,35793,a55475b1,None,None,PRIVATE_SECTOR_EMPLOYEE,88000.0,52.380822,0


In [206]:
person_1_selected = person_1_selected.join(pd.get_dummies(person_1_selected['familystate_447L']))
person_1_selected.drop(columns='familystate_447L', inplace=True)
person_1_selected.head(2)

,case_id,education_927M,empl_industry_691L,incometype_1044T,mainoccupationinc_384A,age,emplyment_period_cat,DIVORCED,LIVING_WITH_PARTNER,MARRIED,SINGLE,WIDOWED
0,0,P97_36_170,OTHER,SALARIED_GOVT,10800.0,34.287671,3,False,False,True,False,False
4,1,P97_36_170,OTHER,SALARIED_GOVT,10000.0,63.221918,3,True,False,False,False,False


In [207]:
person_1_selected['education_927M'].unique()

array(['P97_36_170', 'P33_146_175', 'P17_36_170', 'P106_81_188',
       'P157_18_172', 'a55475b1'], dtype=object)

In [208]:
person_1_selected['incometype_1044T'].unique()

array(['SALARIED_GOVT', 'EMPLOYED', 'PRIVATE_SECTOR_EMPLOYEE',
       'RETIRED_PENSIONER', 'SELFEMPLOYED', 'HANDICAPPED_2', 'OTHER',
       'HANDICAPPED_3', 'HANDICAPPED'], dtype=object)

In [209]:
person_1_selected['income_group'] = 'No_Income'
person_1_selected.loc[person_1_selected['incometype_1044T'].isin(['SALARIED_GOVT', 'EMPLOYED', 'PRIVATE_SECTOR_EMPLOYEE', 'SELFEMPLOYED']), 'income_group'] = 'Salaried_Income'
person_1_selected.loc[person_1_selected['incometype_1044T'].isin(['RETIRED_PENSIONER']), 'income_group'] = 'Other_Income'
person_1_selected = person_1_selected.join(pd.get_dummies(person_1_selected['income_group']))
person_1_selected.drop(columns=['incometype_1044T', 'education_927M', 'empl_industry_691L', 'income_group'], inplace=True)
person_1_selected.head()

,case_id,mainoccupationinc_384A,age,emplyment_period_cat,DIVORCED,LIVING_WITH_PARTNER,MARRIED,SINGLE,WIDOWED,No_Income,Other_Income,Salaried_Income
0,0,10800.0,34.287671,3,False,False,True,False,False,False,False,True
4,1,10000.0,63.221918,3,True,False,False,False,False,False,False,True
9,2,14000.0,45.876712,3,False,False,True,False,False,False,False,True
14,3,10000.0,27.197260,3,False,False,True,False,False,False,False,True
17,4,24000.0,26.778082,3,False,False,True,False,False,False,False,True


Other info

In [210]:
other = pd.read_parquet('../data/raw/train_other_1.parquet')
other.head()

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
0,43801,12466.6010,12291.2000,914.2000,0.0,304.80002,0
1,43991,3333.4001,3273.4001,0.0000,0.0,0.00000,0
2,44001,10000.0000,10000.0000,0.0000,0.0,0.00000,0
3,44053,0.0000,0.0000,2586.4001,0.0,88.80000,0
4,44130,63.8000,60.8000,0.0000,0.0,0.00000,0


In [211]:
features[features['Variable'].isin(other.columns)]

,Variable,Description
9,amtdebitincoming_4809443A,Incoming debit card transactions amount.
10,amtdebitoutgoing_4809440A,Outgoing debit card transactions amount.
11,amtdepositbalance_4809441A,Deposit balance of client.
12,amtdepositincoming_4809444A,Amount of incoming deposits to client's account.
13,amtdepositoutgoing_4809442A,Amount of outgoing deposits from client's acco...


In [212]:
other[other['num_group1'] > 0]

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1


In [213]:
other_grouped = other.groupby('case_id').mean().reset_index()
other_grouped.head(2)

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
0,43801,12466.6010,12291.2000,914.2,0.0,304.80002,0.0
1,43991,3333.4001,3273.4001,0.0,0.0,0.00000,0.0


In [214]:
other_grouped.drop(columns=['num_group1'], inplace=True)

Merging all frames here into one.

In [215]:
from functools import reduce

In [216]:
second_level_data_1 = reduce(lambda a, b: pd.merge(left=a, right=b, on='case_id', how='outer'), [debitcard_mean, deposit_mean, person_1_selected, other_grouped])
second_level_data_1.head()

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,debitcard_days,amount_416A,contract_days,mainoccupationinc_384A,age,emplyment_period_cat,DIVORCED,LIVING_WITH_PARTNER,MARRIED,SINGLE,WIDOWED,No_Income,Other_Income,Salaried_Income,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
0,0,NaN,NaN,NaN,NaN,NaN,10800.0,34.287671,3,False,False,True,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,10000.0,63.221918,3,True,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,14000.0,45.876712,3,False,False,True,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,10000.0,27.197260,3,False,False,True,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,24000.0,26.778082,3,False,False,True,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN


In [217]:
second_level_data_1.to_parquet('../data/processed/second_level_1.parquet.gz', compression='gzip')